In [ ]:
import os
import zipfile
import shutil
import numpy as np
import tensorflow as tf
import glob
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [ ]:
dataset_dir = "./gerador-mensagens/database_face"
extract_path = f"{dataset_dir}/images"
zip_path = f"{dataset_dir}/facial-emotion-expressions.zip"
path_train_path = f'{dataset_dir}/images/train'
path_validation_path = f'{dataset_dir}/images/validation'

In [ ]:
os.makedirs(dataset_dir, exist_ok=True)

if not os.path.exists(zip_path):
    !curl -L -o {zip_path} https://www.kaggle.com/api/v1/datasets/download/samaneheslamifar/facial-emotion-expressions

if not os.path.exists(extract_path):
    zipfile.ZipFile(zip_path, 'r').extractall(dataset_dir)

classes = os.listdir(path_train_path)
for folder in classes:
    if folder not in ["happy", "sad"]:
        shutil.rmtree(os.path.join(path_train_path, folder))
        shutil.rmtree(os.path.join(path_validation_path, folder))

if os.path.exists(f"{extract_path}/images"):
    shutil.rmtree(f"{extract_path}/images")

In [ ]:
database_train = image_dataset_from_directory(
    path_train_path,
    image_size=(48, 48),
    batch_size=32,
    label_mode="categorical",
    color_mode="grayscale",
    seed=123,
    shuffle=True
).map(lambda x, y: (x / 255.0, y))

database_validation = image_dataset_from_directory(
    path_validation_path,
    image_size=(48, 48),
    batch_size=32,
    label_mode="categorical",
    color_mode="grayscale",
    seed=123,
    shuffle=True
).map(lambda x, y: (x / 255.0, y))

model = models.Sequential([
    layers.InputLayer(input_shape=(48, 48, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(database_train, validation_data=database_validation, epochs=10)


In [ ]:
model.save('./consumidor-sentimento/model.h5',  include_optimizer=False) 

# Fotos trainos de futebol

In [4]:
dataset_dir = "./gerador-mensagens/database_futebol"

In [9]:
if not os.path.exists(f"{dataset_dir}/futebol_logos.zip"):
    os.makedirs(dataset_dir, exist_ok=True)
    !curl -L -o {dataset_dir}/futebol_logos.zip https://github.com/luukhopman/football-logos/archive/refs/heads/master.zip

if not os.path.exists(f"{dataset_dir}/database_futebol"):
    zipfile.ZipFile(f"{dataset_dir}/futebol_logos.zip", 'r').extractall(dataset_dir)
    os.rename(f"{dataset_dir}/football-logos-master", f"{dataset_dir}/database_futebol")
    

In [ ]:


base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
embedding_model = Model(inputs=base_model.input,outputs=tf.keras.layers.GlobalAveragePooling2D()(base_model.output))

path_futebol_logos = glob.glob(f"{dataset_dir}/database_futebol/logos/*/*.png")

imgs = []
labels = []

for img_path in path_futebol_logos:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    imgs.append(img_array)
    
    label = os.path.basename(os.path.dirname(img_path))
    labels.append(label)

imgs = np.array(imgs)
imgs = preprocess_input(imgs)
embeddings = embedding_model.predict(imgs, batch_size=32, verbose=1)
labels = np.array(labels)

print("Embeddings shape:", embeddings.shape)
print("Labels shape:", labels.shape)

13/13 [==============================] - 5s 303ms/step
Embeddings shape: (398, 1280)
Labels shape: (398,)


In [21]:
np.savez_compressed(f"consumidor-times/futebol_embeddings_labels.npz", embeddings=embeddings, labels=labels)

In [20]:
from numpy.linalg import norm
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

def predict(embedding, embeddings_train, labels_train):
    sims = [cosine_similarity(embedding, e) for e in embeddings_train]
    return labels_train[np.argmax(sims)]

data = np.load(f"{dataset_dir}/futebol_embeddings_labels.npz")
embeddings = data['embeddings']
labels = data['labels']
y_pred = [predict(e, embeddings, labels) for e in embeddings]

accuracy = np.mean(np.array(y_pred) == labels)
print(f"Training accuracy: {accuracy * 100:.2f}%")

Training accuracy: 100.00%
